In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Input, Dense, Flatten, Concatenate
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os

In [2]:
np.random.seed(10)

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


In [9]:
class Random_Finetune_ResNet50():
    def __init__(self, input_shape):

        self.fitness = 0
        
        IMG_SHAPE = input_shape + (3,)
        self.base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
        sample_arr = [True, False]
        self.bool_arr = np.random.choice(sample_arr, size=len(self.base_model.layers))
        self.base_model.trainable = True
        for idx, i in enumerate(self.base_model.layers):
            i.trainable = self.bool_arr[idx]
        
    def forward(self, learning_rate=0.001):
        inputs = Input((28, 28, 1))
        resized_x = tf.keras.layers.experimental.preprocessing.Resizing(32, 32)(inputs)
        first_conv_layer = Conv2D(3, 1, padding='same', activation=None)(resized_x)

        x = self.base_model(first_conv_layer, training = False)
        x = Flatten()(x)
        outputs = Dense(10, activation = 'softmax')(x)

        model = tf.keras.Model(inputs, outputs, name="fashion_mnist_resnet50_model")

        # 'categorical_crossentropy'은 y[0]=[0, 0, 0, 0, 0, 0, 0, 0, 1], y[1, 0, 0, 0, 0, 0, 0, 0, 0]과 같이 one-hot-encoding label일 경우에 사용
        model.compile(loss="categorical_crossentropy", 
        optimizer=tf.keras.optimizers.Adam(learning_rate= learning_rate), 
        metrics=['accuracy'])
        
        return model
    
    def train_model(self, model, train_data, train_targets, validation_data=(x_test, y_test), epochs=20, batch_size=256):
    
        early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        checkpoint_best_path = 'model_checkpoints_best/checkpoint'
        checkpoint_best = ModelCheckpoint(filepath=checkpoint_best_path,
                                        save_weights_only=True,
                                        save_freq='epoch',
                                        monitor='val_accuracy',
                                        save_best_only=True,
                                        verbose=0)
        history = model.fit(train_data, train_targets,
                        validation_data = validation_data,
                        epochs = epochs,
                        batch_size = batch_size,
                        verbose = 1,
                        callbacks=[early])
        return history

In [5]:
N_POPULATION = 10
N_BEST = 5
N_CHILDREN = 5
PROB_MUTATION = 0.2

In [6]:
genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(N_POPULATION)]
best_genomes = None

In [7]:
(genomes[0].bool_arr ==genomes[1].bool_arr).all()

False

In [10]:
n_gen = 0
while True:
  n_gen += 1
  for i, genome in enumerate(genomes):
      model = genome.forward(0.0001)
    #   print(genome.bool_arr)
      history = genome.train_model(model, x_train, y_train, (x_test, y_test), 10, 256)
      fitness = history.history['val_accuracy']
      score = history.history['val_loss']
      
      fitness.sort()
      score.sort()
      genome.fitness = fitness[-1]
      score = score[0]

      print('Generation #%s, Genome #%s, Fitness: %s, Score: %s' % (n_gen, i, genome.fitness, score))
  if best_genomes is not None:
      genomes.extend(best_genomes)
  genomes.sort(key=lambda x: x.fitness, reverse=True)

  print('===== Generaton #%s\tBest Fitness %s =====' % (n_gen, genomes[0].fitness))
  break

Epoch 1/10
235/235 [==============================] - 22s 72ms/step - loss: 0.4187 - accuracy: 0.8509 - val_loss: 0.3448 - val_accuracy: 0.8728
Epoch 2/10
235/235 [==============================] - 16s 66ms/step - loss: 0.3156 - accuracy: 0.8828 - val_loss: 0.3674 - val_accuracy: 0.8624
Epoch 3/10
235/235 [==============================] - 16s 66ms/step - loss: 0.3049 - accuracy: 0.8861 - val_loss: 0.3482 - val_accuracy: 0.8692
Epoch 4/10
235/235 [==============================] - 16s 67ms/step - loss: 0.2958 - accuracy: 0.8881 - val_loss: 0.3562 - val_accuracy: 0.8615
Epoch 5/10
235/235 [==============================] - 16s 66ms/step - loss: 0.2795 - accuracy: 0.8943 - val_loss: 0.3175 - val_accuracy: 0.8850
Epoch 6/10
235/235 [==============================] - 16s 66ms/step - loss: 0.2751 - accuracy: 0.8967 - val_loss: 0.2873 - val_accuracy: 0.8960
Epoch 7/10
235/235 [==============================] - 16s 66ms/step - loss: 0.2642 - accuracy: 0.8998 - val_loss: 0.3195 - val_accuracy:

KeyboardInterrupt: 